In [2]:
from google.colab import files
uploaded = files.upload()

Saving pos.txt to pos.txt


In [3]:
from google.colab import files
uploaded = files.upload()

Saving negative.txt to negative.txt


In [5]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd

pos_revs = open("pos.txt")
neg_revs = open("negative.txt")

pos = pos_revs.readlines()
neg = neg_revs.readlines()

df = pd.DataFrame(columns=['sentence', 'polarity'])
df['sentence'] = neg + pos
df['polarity'] =  [np.array([0,1])]*len(neg) + [np.array([1,0])]*len(pos)
df = df.sample(frac=1, random_state=10) # Shuffle the rows
df.reset_index(inplace=True, drop=True)

df.head()



,sentence,polarity
0,That means the actual capacity is more like 20...,"[0, 1]"
1,My only problem is that it is MADE IN CHINA.\n,"[0, 1]"
2,The box that the item arrived in was kind of m...,"[0, 1]"
3,It needed you to press fairly hard to register...,"[1, 0]"
4,Now my makeup stays on longer and oil no longe...,"[0, 1]"


In [0]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

word_seq = [text_to_word_sequence(sent) for sent in df['sentence']]

max_sent_length= np.percentile([len(seq) for seq in word_seq], 90)


tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts([' '.join(seq[:int(max_sent_length)]) for seq in word_seq])

X = tokenizer.texts_to_sequences([' '.join(seq[:int(max_sent_length)]) for seq in word_seq])
X = pad_sequences(X, maxlen=int(max_sent_length), padding='post', truncating='post')



In [0]:
X = tokenizer.texts_to_sequences([' '.join(seq[:int(max_sent_length)]) for seq in word_seq])
X = pad_sequences(X, maxlen=int(max_sent_length), padding='post', truncating='post')


In [8]:
from sklearn.model_selection import train_test_split
y=df['polarity'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=10, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=10, test_size=0.1)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)



(648000, 23)
(80000, 23)
(72000, 23)


In [9]:
y_train
y_train = np.array([out for out in y_train])
y_test = np.array([out for out in y_test])
y_val = np.array([out for out in y_val])

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(648000, 2)
(72000, 2)
(80000, 2)


In [10]:
pip install -U gensim

     |████████████████████████████████| 24.2MB 1.2MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [11]:
import gensim

embeddings = gensim.models.Word2Vec(word_seq, min_count = 10,  size = 300)
print("Done....")

Done....


In [12]:
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1,300)) 

for word, i in tokenizer.word_index.items(): # i=0 is the embedding for the zero padding
    try:
        embeddings_vector = embeddings[word]
    except KeyError:
        embeddings_vector = None
    if embeddings_vector is not None:
        embeddings_matrix[i] = embeddings_vector
        
del embeddings

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [14]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional,Flatten
from keras.regularizers import l2

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=300,input_length=int(max_sent_length),
                          weights = [embeddings_matrix], trainable=True, name='word_embedding_layer'))


model.add(Flatten(name='flatten_layer'))
model.add(Dense(32,activation='relu' ,kernel_regularizer=l2(0.010), bias_regularizer=l2(0.010)))

#model.add(Dense(32,activation='tanh',name='hidden_layer'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax',name='output_layer'))

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, 23, 300)           27914700  
_________________________________________________________________
flatten_layer (Flatten)      (None, 6900)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                220832    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
output_layer (Dense)         (None, 2)                 66        
Total params: 28,135,598
Trainable params: 28,135,598
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0628 00:45:42.331055 140079770318720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 00:45:42.365272 140079770318720 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
import numpy as np
model.fit(X_train,y_train ,
          batch_size=1024,
          epochs=10,
          validation_data=(X_val,y_val))

Train on 648000 samples, validate on 80000 samples
Epoch 1/10
648000/648000 [==============================] - 316s 488us/step - loss: 0.6038 - acc: 0.7581 - val_loss: 0.5153 - val_acc: 0.7863
Epoch 2/10
648000/648000 [==============================] - 314s 485us/step - loss: 0.5157 - acc: 0.7855 - val_loss: 0.4924 - val_acc: 0.7958
Epoch 3/10
648000/648000 [==============================] - 314s 485us/step - loss: 0.4957 - acc: 0.7948 - val_loss: 0.4884 - val_acc: 0.7968
Epoch 4/10
648000/648000 [==============================] - 315s 485us/step - loss: 0.4843 - acc: 0.8000 - val_loss: 0.4728 - val_acc: 0.8032
Epoch 5/10
648000/648000 [==============================] - 314s 485us/step - loss: 0.4752 - acc: 0.8042 - val_loss: 0.4687 - val_acc: 0.8068
Epoch 6/10
648000/648000 [==============================] - 314s 485us/step - loss: 0.4688 - acc: 0.8069 - val_loss: 0.4669 - val_acc: 0.8063
Epoch 7/10
648000/648000 [==============================] - 315s 486us/step - loss: 0.4629 - acc:

In [17]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=1024)
print("Accuracy on Test Set = {0:4.3f}".format(acc))


72000/72000 [==============================] - 1s 20us/step
Accuracy on Test Set = 0.806
